In [10]:

pip install alpha_vantage

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install quandl

Note: you may need to restart the kernel to use updated packages.


In [21]:
from alpha_vantage.timeseries import TimeSeries
import quandl
import config


In [22]:

# Initialize the TimeSeries object with your API key
ts = TimeSeries(key=config.api_key, output_format='pandas')

# Define the symbol you want to fetch data for
symbol = 'VEQT.TO'

# Fetch daily historical stock data
data, meta_data = ts.get_daily(symbol=symbol, outputsize='compact')

# Print the data (data is a pandas DataFrame)
print(data.head())

            1. open  2. high  3. low  4. close  5. volume
date                                                     
2023-09-29    34.69    34.74   34.50     34.57    57324.0
2023-09-28    34.24    34.57   34.23     34.50    93500.0
2023-09-27    34.45    34.50   34.11     34.28    80000.0
2023-09-26    34.62    34.62   34.32     34.35   114800.0
2023-09-25    34.59    34.74   34.53     34.70    97100.0
